In [1]:
example = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
""".strip().splitlines()

with open("day11.txt", "r") as f:
    data = f.readlines()

In [12]:
class Square(object):
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def step(self, area: "Area", policy: "Policy") -> "Square":
        return self

In [13]:
class Floor(Square):
    @property
    def occupied(self) -> bool:
        return False

    def __str__(self) -> str:
        return "."

In [14]:
class Seat(Square):
    def __init__(self, x: int, y: int, occupied: bool = False):
        super().__init__(x, y)
        self.occupied = occupied

    def step(self, area: "Area", policy: "Policy") -> "Seat":
        occupied_adjacent = sum(1 for seat in policy.consider_seats(area, self) if seat.occupied)

        if not self.occupied and occupied_adjacent == 0:
            area.stable = False
            return Seat(self.x, self.y, occupied=True)
        elif self.occupied and occupied_adjacent > policy.required_space:
            area.stable = False
            return Seat(self.x, self.y, occupied=False)
        
        return self

    def __str__(self) -> str:
        return "#" if self.occupied else "L"

In [16]:
class Policy(object):
    def __init__(self, consider_seats, required_space: int = 4):
        self.consider_seats = consider_seats
        self.required_space = required_space

part1_policy = Policy(lambda area, seat: area.get_adjacent(seat), required_space=4)
part2_policy = Policy(lambda area, seat: area.get_adjacent_visible(seat), required_space=5)

In [25]:
from typing import List, Iterator, Union


class Area(object):
    def __init__(self, seed: List[str]):
        self.stable = False
        self.grid = [
            [
                Floor(x, y) if seat == '.' else Seat(x, y, occupied=seat == '#')
                for x, seat in enumerate(row.strip())
            ]
            for y, row in enumerate(seed)
        ]

    def get_adjacent(self, square: Square) -> Iterator[Square]:
        d = [-1, 0, 1]

        return (s for s in [
            self.get(square.x + dx, square.y + dy)
            for dx in d
            for dy in d
        ] if s is not None)

    def get_adjacent_visible(self, square: Square, probe_depth: int = 20) -> Iterator[Square]:
        d = [-1, 0, 1]
        
        return  (s for s in [
            next((ns for ns in (self.get(square.x + dx * factor, square.y + dy * factor) for factor in range(1, probe_depth)) if ns is None or isinstance(ns, Seat)), None)
            for dx in d
            for dy in d
        ] if s is not None)

    def get(self, x: int, y: int) -> Union[Square, None]:
        if x < 0 or y < 0:
            return None

        if y >= len(self.grid) or x >= len(self.grid[y]):
            return None

        return self.grid[y][x]

    def filter(self, predicate) -> Iterator[Square]:
        for row in self.grid:
            for square in row:
                if predicate(square):
                    yield square

    def step(self, policy: Policy) -> bool:
        self.stable = True

        self.grid = [
            [square.step(self, policy) for square in row]
            for row in self.grid
        ]

        return not self.stable

    def __str__(self):
        return "\n".join(
            "".join(str(square) for square in row)
            for row in self.grid
        )



test_area = Area(example)
while test_area.step(part1_policy):    
    pass

print(f"Occupied Seats (P1): {sum(1 for seat in test_area.filter(lambda s: s.occupied))}")


test_area = Area(example)
while test_area.step(part2_policy):
    pass

print(f"Occupied Seats (P2): {sum(1 for seat in test_area.filter(lambda s: s.occupied))}")

Occupied Seats (P1): 37
Occupied Seats (P2): 26


In [26]:
true_area = Area(data)
while true_area.step(part1_policy):
    pass

print(f"Occupied Seats (P1): {sum(1 for seat in true_area.filter(lambda s: s.occupied))}")

Occupied Seats (P1): 2338


In [27]:
true_area = Area(data)
while true_area.step(part2_policy):
    pass

print(f"Occupied Seats (P2): {sum(1 for seat in true_area.filter(lambda s: s.occupied))}")

Occupied Seats (P2): 2134
